In [2]:
!nvidia-smi

Wed Dec  2 01:45:37 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 410.104      Driver Version: 410.104      CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    43W / 300W |    417MiB / 16130MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [3]:
#Check that tensorflow recognizes the GPU
import tensorflow as tf
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Cannot recognize GPU")

/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

Default GPU Device: /device:GPU:0


In [4]:
def regex_filter(df):
    df["final_utterance"] = df["final_utterance"].str.lower()

    regex_patterns={
      # replace "uh.," "um.," "ah.," "mm.,", "oh.,!?" with blank but we leave uh-hum/mm-hmm in there
      r'(\buh(\,|\.|\s|\?))': ' ',
      r'(\bum(\,|\.|\s|\?))': ' ',
      r'(\bah\W)': ' ',
      r'(\bmm(\,|\.|\s|\!|\?))': ' ',
      r'(\boh(\,|\.|\s|\!|\?))': ' ',

      r'(\s*(?:\+?(\d{1,3}))?[-. (]*(\d{3})?[-. )]*(\d{3})[-. ]*(\d{4})(?: *x(\d+))?\s*)': ' ', # remove 10 digit and 7 digit numbers  e.g., 456-123-0000 and 123-0000
      r'(\d{3,})': '', # remove any numbers that has length 3 or more 
      r'\b(\w+)((?:\W+?)(\s+)?\1\b)+': r'\1', # replaces duplicate words with a single instance of that word. "yup, yup thank thank you. bye-bye" -> "yup thank you. bye" 
      r'([a-zA-Z0-9._-]+(\s+)?@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)': '[email]',
      r'(\bok\b)': 'okay',  #replaces "ok" with "okay"
      r'(ma\'am)': 'madam', #replaces ma'am with madam 
      r'(\s{2,})': ' '#replaces 2 or more whitespaces with a single one
    }

    df['final_utterance'].replace(regex=regex_patterns, inplace=True)
    df['final_utterance'] = df['final_utterance'].str.strip() 

    return df

In [5]:
from google.cloud import bigquery
import os
import pandas as pd
import numpy as np
import sentencepiece as spm
%pwd

'/home/jupyter/xlnet_pretrain'

In [6]:
project_id = 'talkiq-data'
client = bigquery.Client(project=project_id)

query = """
SELECT call_id, time_end, final_utterance
FROM `talkiq-data.ai_research.dialpad_xlnet_lm_pretrain`
"""

df = client.query(query).to_dataframe() 
df = regex_filter(df)

df.head()

,call_id,time_end,final_utterance
0,5666280148631552,444.508,yeah try and have them click on option number ...
1,5666280148631552,431.395,okay.
2,5666280148631552,456.685,"okay, and then can you just double check the o..."
3,5666280148631552,462.058,mm-hmm so yup hurting good pr pse tendered.
4,5666280148631552,466.348,your peer.


In [128]:
#np.savetxt('train_sentencepiece.txt', df.final_utterance.values, fmt='%s')

## Pre-process data for pretraining an XLNet model

In [7]:
# df_temp = df.groupby(['call_id'])[['time_end']].agg(np.max).reset_index().rename(columns={'time_end':'max_timeend'})
# df_processed = df.join(df_temp.set_index('call_id'), on='call_id')
# df_processed.loc[(df_processed['time_end'] == df_processed['max_timeend']), ['final_utterance']] = df_processed.final_utterance + " \n"

call_ids=df.call_id
unequal = call_ids.index[call_ids.shift(-1) != call_ids ]
df['final_utterance'].loc[unequal] += ' \n'

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
#np.savetxt('lm_pretrain.txt', df.final_utterance.values, fmt='%s')

In [ ]:
#!python xlnet/data_utils.py --use_tpu=False --bsz_per_host=4 --num_core_per_host=1 --uncased=True --seq_len=512 --reuse_len=256 --input_glob="lm_pretrain.txt" --save_dir='./dialpad_xlnet_4/' --sp_path='./dialpad_xlnet_4/spiece.model' --mask_alpha=6 --mask_beta=1 --num_predict=85 --bi_data=True

In [8]:
#bi_data = True
#spiece.model in this folder is the original one used by the authors 
!python xlnet/data_utils.py --use_tpu=False --vocab_size=32000 --bsz_per_host=4 --num_core_per_host=1 --uncased=True --seq_len=512 --reuse_len=256 --input_glob="lm_pretrain.txt" --save_dir='./dialpad_xlnet_32000/' --sp_path='./dialpad_xlnet_32000/spiece.model' --mask_alpha=6 --mask_beta=1 --num_predict=85 --bi_data=True


/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

## XLNet Model Pretrain

In [ ]:
#Though the output is frozen at 98000 training steps, the latest model is savead at checkpoint:
#100,000
#110,000
#

!python xlnet/train_gpu.py --corpus_info_path="./dialpad_xlnet_32000/corpus_info.json" --model_dir="./dialpad_xlnet_32000/model/" --record_info_dir="./dialpad_xlnet_32000/tfrecords/" --use_tpu=False --train_batch_size=4 --num_core_per_host=1 --vocab_size=32000 --seq_len=512 --reuse_len=256 --mem_len=384 --perm_size=256 --untie_r=True --mask_alpha=6 --mask_beta=1 --num_predict=85 --train_steps=200000 --iterations=1000 --save_steps=10000 --uncased=True --bi_data=True --d_head=64 --d_inner=3072 --d_model=768 --ff_activation=gelu --n_head=12 --n_layer=12 --learning_rate=3e-4


/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

## Convert pretrained XLNet model from TF to Pytorch

In [24]:
!pwd

/home/jupyter/xlnet_pretrain


In [ ]:
# !pip install transformers
# !pip install torch

In [ ]:
"/home/jupyter/xlnet_pretrain/dialpad_xlnet_bi4/model/model.ckpt"

In [27]:
from transformers import XLNetTokenizer, TFXLNetForSequenceClassification
import tensorflow as tf

/opt/conda/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10000). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [28]:
model = TFXLNetForSequenceClassification.from_pretrained('/home/jupyter/xlnet_pretrain/dialpad_xlnet_bi4/model/', from_tf=True)

ImportError: 
TFXLNetForSequenceClassification requires the TensorFlow library but it was not found in your environment. Checkout the instructions on the
installation page: https://www.tensorflow.org/install and follow the ones that match your environment.


In [26]:
# export TRANSFO_XL_CHECKPOINT_PATH=/path/to/xlnet/checkpoint
# export TRANSFO_XL_CONFIG_PATH=/path/to/xlnet/config

!transformers-cli convert --model_type xlnet --tf_checkpoint "/home/jupyter/xlnet_pretrain/dialpad_xlnet_bi4/model/model.ckpt" --config "/home/jupyter/xlnet_pretrain/dialpad_xlnet_bi4/model/config.json" --pytorch_dump_output "/home/jupyter/xlnet_pretrain/dialpad_xlnet_bi4/pytorch_model"



/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 